# How has the mainstream media's coverage of Covid-19 changed during the pandemic?
- ### Can an overall change in sentiment be seen?
- ### Has the topics changed over time?
- ### Can a change in positivity og negativity within publishers be seen?

In [1]:
import os.path
# Retrieves the dataset from https://www.kaggle.com/jannalipenkova/covid19-public-media-dataset
!kaggle datasets download -d jannalipenkova/covid19-public-media-dataset

covid19-public-media-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
from zipfile import ZipFile

# Creates a ZipFile Object and loads the dataset into it
with ZipFile('covid19-public-media-dataset.zip', 'r') as zipObj:
    # Extracts all the contents of the zip file in the current directory
    zipObj.extractall('datasets')

In [3]:
import pandas as pd
pd.set_option('max_colwidth', 150)

data_df = pd.read_csv('datasets/covid19_articles_20200504.csv', index_col=0)

data_df

,title,date,domain,url,author,content,topic_area
0,My experience of surviving cancer twice,2020-01-03,medicalnewstoday,https://www.medicalnewstoday.com/articles/327373,Helen Ziatyk,"“Helen, I’m so sorry to tell you that you have stage 4 ovarian cancer.” I will never forget hearing those words. Cancer treatment was pretty gruel...",healthcare
1,Ginger: Health benefits and dietary tips,2020-01-03,medicalnewstoday,https://www.medicalnewstoday.com/articles/265990.php,Jenna Fletcher,"If you buy something through a link on this page, we may earn a small commission. How this works. People have used ginger in cooking and medicine ...",healthcare
2,China pneumonia outbreak may be caused by Sars-type virus: WHO | Science | The Guardian,2020-01-08,theguardian,https://www.theguardian.com/science/2020/jan/09/china-pneumonia-outbreak-may-be-caused-by-sars-type-virus-who,Reuters,A cluster of more than 50 pneumonia cases in the central Chinese city of Wuhan may be due to a newly emerging member of the family of viruses that...,general
3,New virus identified as likely cause of mystery illness in China,2020-01-08,nature,https://www.nature.com/articles/d41586-020-00020-9,nature,Passengers arriving at Hong Kong's international airport are being monitored for signs a mystery illness that emerged in central China. Credit: An...,science
4,China's Sars-like illness worries health experts | World news | The Guardian,2020-01-09,theguardian,https://www.theguardian.com/world/2020/jan/09/chinas-sars-like-illness-worries-health-experts,https://www.theguardian.com/profile/sarahboseley,"The finding that the outbreak of viral pneumonia in China that has struck 59 people may be caused by a coronavirus, the family of viruses behind S...",general
...,...,...,...,...,...,...,...
55532,'It was only advice' Dr Hilary rejects blanket elderly lockdown as he issues stark warning | TV & Radio | Showbiz & TV | Express.co.uk,2020-05-04,express,https://www.express.co.uk/showbiz/tv-radio/1277406/ITV-GMB-Dr-Hilary-Jones-coronavirus-over-70s-death-rate-COVID-19-latest,Aurora Bosotti,"Good Morning Britain's Dr Hilary Jones said the Government had only ever ""advised"" elderly Britons to self-isolate but conceded no blanket order ...",general
55533,Liverpool star Divock Origi provides update on his future amid Timo Werner transfer talk | Football | Sport | Express.co.uk,2020-05-04,express,https://www.express.co.uk/sport/football/1277408/Liverpool-Divock-Origi-Timo-Werner-transfer-news,Mikael McKenzie,Liverpool forward Divock Origi insists his future lies at Anfield despite talk that Jurgen Klopp is considering a swoop for RB Leipzig star Timo ...,general
55534,New York: US states join forces to buy vital medical equipment | World | News | Express.co.uk,2020-05-04,express,https://www.express.co.uk/news/world/1277388/new-york-us-states-ppe-medical-equipment-coronavirus-latest-donald-trump-news-cuomo,Melanie Kaidan,"The deal was agreed by Connecticut, Delaware, Massachusetts, New Jersey, New York, Pennsylvania and Rhode Island. The adjacent states have also a...",general
55535,Coronavirus US: Millions more make joblessness claims | City & Business | Finance | Express.co.uk,2020-05-04,express,https://www.express.co.uk/finance/city/1277374/jobs-unemployment-lockdown-claims-us-coronavirus,Dylan Donnelly,Businesses around the world have seen major losses and redundancies amid the pandemic’s mass shut down of public life. Figures released this week...,general


In [ ]:
# Apply first round of text cleaning
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
data_clean = pd.DataFrame(data_df.content.apply(round1))
data_clean

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
data_clean = pd.DataFrame(data_clean.content.apply(round2))
data_clean

In [ ]:
import nltk, re

def clean_text_round3(text):
    
    covid_reg = re.compile(r'.*corona.*|.*covid.*', re.IGNORECASE)
    
    while True:
        try:
            words = set(nltk.corpus.words.words())
        except:
            nltk.download('words')
        break

    return " ".join(w for w in nltk.wordpunct_tokenize(text) \
             if w.lower() in words or covid_reg.match(w))

round3 = lambda x: clean_text_round3(x)

In [ ]:
#data_clean = pd.DataFrame(data_clean.content.apply(round3))

#if not os.path.exists("pickle/"):
#    !mkdir 'pickle'
#data_clean.to_pickle('pickle/data_clean_r3.pkl')

In [4]:
if not os.path.exists("pickle/"):
    !mkdir 'pickle'
data_clean = pd.read_pickle('pickle/data_clean_r3.pkl')

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

def clean_text_round4(text):

    lemmer=WordNetLemmatizer()
    
    return ' '.join([lemmer.lemmatize(word) for word in nltk.wordpunct_tokenize(text) if word not in 'was'])

round4 = lambda x: clean_text_round4(x)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/berantzino/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
data_clean = pd.DataFrame(data_clean.content.apply(round4))

In [7]:
# Pickles the cleaned data
if not os.path.exists("pickle/"):
    !mkdir 'pickle'
data_clean.to_pickle('pickle/data_clean_r4.pkl')
data_df.to_pickle('pickle/data_df.pkl')